<h3 style="color:orange">Load Necessary Library</h3>


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import datasets, layers, models

from sklearn.metrics import classification_report

# setting a random seed to reproduce results
seed=4
tf.random.set_seed(4)

## Data Collection

### Load the mnist datasets

In [ ]:
mnist = tf.keras.datasets.mnist

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

X_train = np.concatenate((X_train, X_test))
y_train = np.concatenate((Y_train, Y_test))
y_train = y_train.astype(int)

X_train.shape, y_train.shape

### Read the given kaggle datasets

In [ ]:
valid = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
valid.shape, test.shape

## Data Preprocessing

In [ ]:
y_validation = valid.label.values
x_validation = valid.drop('label', axis = 1).values

### Reshape the features to 28 X 28 shape

In [ ]:
x_train = X_train.reshape(X_train.shape[0], 28,28)
x_validation = x_validation.reshape(x_validation.shape[0],28,28)
x_test = test.values.reshape(test.shape[0], 28,28)

x_train.shape, y_train.shape, x_validation.shape, y_validation.shape,  x_test.shape

### Plot the first 50 digit with their actual label

In [ ]:
plt.figure(figsize=(20,7), dpi = 523)
for i in range(30):
    plt.subplot(3,10,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(True)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(y_train[i])                   # Adding label as the value of handwritten digit
plt.show()

#### Add a channels dimension

In [ ]:
x_train = x_train.reshape(-1,28,28,1)
x_valid = x_validation.reshape(-1,28,28,1)
x_test  = x_test.reshape(-1,28,28,1)

print('Train Data shape      :',x_train.shape)
print('Validation Data shape :',x_valid.shape)
print('Test Data shape       :',x_test.shape) 

#### Scalling the value for better accuracy

In [ ]:
x_train, x_valid, x_test = x_train / 255, x_valid / 255, x_test / 255

In [ ]:
# One hot-encoding the labels
y_train_ = tf.keras.utils.to_categorical(y_train)
y_valid_ = tf.keras.utils.to_categorical(y_validation)

## Model Building

#### Defining batch size, number epochs (iterations) and the steps in each epoch

In [ ]:
# defining batch size, number epochs (iterations) and the steps in each epoch

batch = 70
epochs = 25

steps_per_epoch = x_train.shape[0]//batch
steps_per_epoch

### Data augmentation

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10,
                             zoom_range=0.15
                            )

# setting data generator to be ready for model

image_generator = datagen.flow(x_train, y_train_, 
                               batch_size=batch, 
                               seed=seed,
                              )

In [ ]:
print('The Unique digits are :', sorted(pd.unique(y_train)))
n = len(pd.unique(y_train))
print('There are total',n, 'unique digits')

### Model

In [ ]:
# Model Bulding
model = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(2, 2), activation='relu', input_shape = (28,28,1)),         # Convolution Layer
    layers.Conv2D(32, (4, 4), activation = 'relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),
    
    layers.Conv2D(filters=64, kernel_size=(2, 2), activation='relu'),         # Convolution Layer
    layers.Conv2D(64, (4, 4), activation = 'relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),                                             # MaxPooling
    
    layers.Conv2D(128, (3, 3), activation = 'relu'),
    layers.BatchNormalization(),
    
    layers.Flatten(),                        
    layers.Dense(150, activation = tf.nn.relu),     # Hidden layer
    layers.Dense(150, activation = tf.nn.relu),      # Hidden layer
    layers.Dense(n, activation = tf.nn.softmax)     # Output layer
])

model.summary()

In [ ]:
# Model Visualised
tf.keras.utils.plot_model(model, show_shapes=True, show_dtype=True, show_layer_names=True, expand_nested=True)

#### Additional parameters to control the training process, avoid overfitting, saving the best model in training, custamize the learning rate etc.

In [ ]:
lr_rate = 0.0003

#Stop training when a monitored metric has stopped improving.
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                  patience = 3,
                                                  min_delta = 1e-4,
                                                  restore_best_weights = True)
# save the Keras model or model weights
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = 'saved_model/best_model_todate', 
                                                 save_best_only = True, 
                                                 save_weights_only = True,
                                                 monitor='val_accuracy',
                                                 mode='max')

# terminates training when a NaN loss is encountered
tn = tf.keras.callbacks.TerminateOnNaN()

'''
A LearningRateSchedule that uses an exponential decay schedule.

When training a model, it is often useful to lower the learning rate as
the training progresses. This schedule applies an exponential decay function
to an optimizer step, given a provided initial learning rate.
'''
scheduler = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate = lr_rate,
                                                           decay_steps = steps_per_epoch//4,
                                                           decay_rate= 0.80,
                                                           staircase=True)
# Learning rate scheduler
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Reduce learning rate when a metric has stopped improving
lr_plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',
                                                  factor = 0.1,
                                                  patience = 4, 
                                                  verbose = 3)

#### Choose an optimizer and loss function for training

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_rate) 

#### Compile the model

In [ ]:
model.compile(optimizer=optimizer,loss=loss_object, metrics=['accuracy'])

#### Train the model

In [ ]:
history = model.fit(image_generator, 
                    epochs=epochs, 
                    validation_data=(x_valid, y_valid_), 
                    steps_per_epoch=steps_per_epoch,
                    callbacks = [lr_scheduler, lr_plateau, checkpoint_callback, early_stopping, tn]
                   )

## Model Evaluation

In [ ]:
loss, accuracy = model.evaluate(x_valid, y_valid_)
print('The accuracy of model on unknown data is',round((accuracy*100),2),'%')

In [ ]:
history_dict = history.history
history_dict.keys()

acc = history_dict['accuracy']                        # Accuracy obtained on training data
val_acc = history_dict['val_accuracy']                # Accuracy obtained on Validation data

loss = history_dict['loss']                                  # Losses obtained on training data
val_loss = history_dict['val_loss']                          # Losses obtained on validation data

#### Training and validation loss

In [ ]:
epochs = range(1, len(acc) + 1)                              # Epoch = range(1, 10+1)  10 iterations        

# Training Loss
plt.plot(epochs, loss, 'ro', label='Training loss')          # "ro" is for "red dot"
# Validation Loss
plt.plot(epochs, val_loss, 'r', label='Validation loss')     # r is for "solid red line"

plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

#### Training and validation accuracy

In [ ]:
# Training accuracy
plt.plot(epochs, acc, 'go', label='Training acc')                 # "go" is for "green dot"
# Validation accuracy
plt.plot(epochs, val_acc, 'g', label='Validation acc')            # g is for "solid green line"
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()

## Predict the Label


In [ ]:
# Predict the label
y_predicted = model.predict(x_valid)

# Get the maximum value integer as the output for the predicted value 
y_predicted_labels = [np.argmax(i) for i in y_predicted]

print('Predicted Label :',y_predicted_labels[:10])
print('Actual Label    :',y_validation[:10])

### Confusion matrix

In [ ]:
cm = tf.math.confusion_matrix(labels = y_validation, predictions=y_predicted_labels)

plt.figure(figsize = (8,5), dpi = 103)
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

### Classification Report

In [ ]:
print("Classification Report: \n", classification_report(y_validation, y_predicted_labels))

### Verify predictions

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    true_label, img = true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'green'
    else:
        color = 'red'
    plt.xlabel("Pred: {} ({:2.0f}%) \n Truth : {}".format(predicted_label, (100*np.max(predictions_array)),
                                                          true_label), color=color)
    
def plot_value_array(i, predictions_array, true_labels):
    true_label = true_labels[i]
    plt.grid(False)
    plt.xticks(range(10))

    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)
    

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 4
num_cols = 4
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows), dpi = 523)
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, y_predicted[i], y_validation, x_valid)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, y_predicted[i], y_validation)
plt.tight_layout()
plt.show()

### Check the wrong predictions

In [ ]:
#predicted_label = np.argmax(predictions_array)
count = 0
wrong_predictions = []
for i in range(len(y_validation)):
    predicted_label = np.argmax(y_predicted[i])
    if predicted_label != y_validation[i]:
        count +=1
        wrong_predictions.append(i)
print('There are',count,'wrong predictions in test datasets')
print('Wrong predicted indices :\n',wrong_predictions)

In [ ]:
num_rows = 4
num_cols = 4
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows), dpi = 723)

for ix in range(num_images):
    i = wrong_predictions[ix]

    plt.subplot(num_rows, 2*num_cols, 2*ix+1)
    plot_image(i, y_predicted[i], y_validation, x_valid)
    plt.subplot(num_rows, 2*num_cols, 2*ix+2)
    plot_value_array(i, y_predicted[i], y_validation)
plt.tight_layout()
plt.show()

## Submission

In [ ]:
predict = model.predict(x_test)
labels = [np.argmax(i) for i in predict]

labels[:5]

In [ ]:
plt.figure(figsize=(20,7), dpi = 523)
for i in range(30):
    plt.subplot(3,10,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(True)
    plt.imshow(x_test[i], cmap=plt.cm.binary)
    plt.xlabel(labels[i])                   # Adding label as the value of handwritten digit
plt.show()

### Submission

In [ ]:
submission = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
submission['Label']=labels
submission.to_csv('submission.csv', index = False)
pd.read_csv('submission.csv')